<a href="https://colab.research.google.com/github/cibercitizen1/cuda_hello/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-yqfsoygc
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-yqfsoygc
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=1b90098fec2f6d41af785690ba3605a4d10f6dbb7fe9eff0497c1b08a3da7a02
  Stored in directory: /tmp/pip-ephem-wheel-cache-xkcky86k/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [3]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [10]:
%%cu
// -------------------------------------------------------------
// -------------------------------------------------------------
#include <cstdio>
#include <iostream>

// -------------------------------------------------------------
// -------------------------------------------------------------
using namespace std;

// -------------------------------------------------------------
// Z -> () -> Z
// -------------------------------------------------------------
__global__ void prueba_kernel(int* p_input, int* p_output) {

  // find out what cell I have to use
  int idx = (blockIdx.x * blockDim.x) + threadIdx.x;

  p_output[idx] = (blockIdx.x * 1000000) + (blockDim.x * 100) + threadIdx.x; + threadIdx.x;
  //p_output[idx] = 200 + p_input[idx];

}

int main() {

  //
  // input and output local arrays
  //
  const int N=128;
  int numbers[N];
  int results[N];

  for (int i = 0; i <= N-1; i++) {
	numbers[i] = i;
	results[i] = -i;
  }


  //
  // input and output arrays in the device (the GPU)
  //
  int* p_in;
  int* p_out;
  int tam = N * sizeof(int);

  cudaMalloc(&p_in, tam);
  cudaMalloc(&p_out, tam);

  //
  // copy to device
  //
  cudaMemcpy(p_in, numbers, tam, cudaMemcpyHostToDevice);

  dim3 total_blocks( 4 );
  dim3 threads_per_block( N/4 );
  
  // dim3 total_blocks( 1 );
  // dim3 threads_per_block( N );

  //
  // timers
  //
  cudaEvent_t start; 
  cudaEvent_t end;
  cudaEventCreate(&start);
  cudaEventCreate(&end);
  
  cudaEventRecord(start);
	
  //
  // start up the kernel(s)
  //
  prueba_kernel<<<total_blocks, threads_per_block>>>(p_in, p_out);

  //
  // wait for completion
  //
  cudaEventRecord(end);
  cudaEventSynchronize(end);

  //
  // copy from device
  //
  cudaMemcpy(&results[0], p_out,  tam, cudaMemcpyDeviceToHost);

  //
  // results
  //
  float time = 0;
  cudaEventElapsedTime(&time, start, end);

  cout << "results[1] : " << results[1] << endl;

  cout << "results[31] : " << results[31] << endl;
  cout << "results[32] : " << results[32] << endl;

  cout << "results[63] : " << results[63] << endl;
  cout << "results[64] : " << results[64] << endl;
  cout << "results[65] : " << results[65] << endl;


  cout << "The time required : ";
  cout << time << endl;
} // main()
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------
// -------------------------------------------------------------

results[1] : 3201
results[31] : 3231
results[32] : 1003200
results[63] : 1003231
results[64] : 2003200
results[65] : 2003201
The time required : 0.01984



In [7]:
!ls


sample_data  src


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
